In [1]:
import torch
import torch.nn.functional as F
import modules_rwkv

In [2]:
w = torch.rand((1024, 124)).to("cuda")
k = torch.rand((1024, 124)).to("cuda")
v = torch.rand((1024, 124)).to("cuda")
r = torch.rand((1024, 124)).to("cuda")
u = torch.rand(124).to("cuda")

In [3]:
#%%timeit
"""
S_t = u A_t + A_{t-1} + w_{t-1} A_{t-2} + w_{t-1} w_{t-2} A_{t-3} + ...

[   1   x   1   ]xA_{t-1}
[   1   xW_{t-1}]xA_{t-2}

[0, 0] [W_{t-2}, W_{t-1}] A_{t-1}
[0, 1] [W_{t-2}, W_{t-1}] A_{t-2}
RWKV注意力，并行计算
"""
a = torch.einsum('...j,...k->...jk', k, v).to("cuda")
wkv = torch.sum(torch.prod(
    torch.tril(
        torch.ones((k.size(-2) - 1, k.size(-2) - 1)).to("cuda"),
        diagonal=-1
    ).flip(-1).unsqueeze(-1).unsqueeze(-4) *
    (w[..., :-1, :] - 1).unsqueeze(-3).repeat(1, k.size(-2) - 1, 1, 1) + 1,
    dim=-2
).unsqueeze(-2) * a[..., :-1, :, :].flip(-3), dim=-3) + u * a[..., -1, :, :]
o1 = r @ wkv
o1 = o1.view(w.shape)

In [4]:
#%%timeit
"""RWKV注意力，循环计算"""
s = torch.zeros_like(u)
for i in range(k.shape[-2]):
    a = torch.einsum('...j,...k->...jk', k[..., i, :], v[..., i, :]).to("cuda")
    wkv = s + u * a
    s = w[..., i, :].unsqueeze(-2) * s + a
o2 = r @ wkv
o2 = o2.view(w.shape)

In [5]:
o1.shape

torch.Size([1024, 124])

In [6]:
o2.shape

torch.Size([1024, 124])

In [7]:
(o1 - o2).max() < 1e-4 # 确保不同方式计算结果一致

tensor(True, device='cuda:0')

In [8]:
from modules_rwkv import *

wkv = WKV(124).to('cuda:0')
wkv(r, w, k ,v).shape

torch.Size([1024, 124])

In [9]:
model = RWKV(10, 10, 5, 2, 2)